<a href="https://colab.research.google.com/github/softip/projeto_IA368X/blob/main/notebooks/Tratamentode_Dados_para_ML_Cox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importação dos Dados**

In [335]:

import pandas as pd

#importa dataset do github
df = []
for i in range(1,10):
  df.append(pd.read_csv("https://raw.githubusercontent.com/softip/projeto_IA368X/main/data/external/dados_covid_ce_{}.csv".format(i)))

dfTotal = pd.DataFrame()
dfTotal = pd.concat(df)
dfTotal.reset_index()

In [336]:
#mostra os campos e a quantidade de dados faltantes
dfTotal.isna().sum()

codigoPaciente                            0
paisPaciente                        1786289
estadoPaciente                          256
codigoMunicipioPaciente               14685
municipioPaciente                        40
sexoPaciente                             52
idadePaciente                            80
dataNotificacao                           0
dataColetaExame                      375130
dataResultadoExame                   379586
resultadoFinalExame                  117663
dataInicioSintomas                    17533
tipoTesteExame                       117983
racaCorPaciente                       95777
municipioNotificacaoEsus              30541
profissionalSaude                     30503
cnesNotificacaoEsus                  385965
idSivep                             1791914
classificacaoEstadoSivep            1805751
dataInternacaoSivep                 1801016
dataEntradaUtisSvep                 1855513
bairroPaciente                        19590
comorbidadeNeurologiaSivep      

In [337]:
#mostra a quantidade de registros no dataset
dfTotal.shape

(1873583, 41)

# **Seleciona apenas os pacientes que testaram positivo para Covid**

In [338]:
dfCasosPositivos = dfTotal.loc[dfTotal.resultadoFinalExame.eq("Positivo")]

In [339]:
#mostra o número total de registros
dfCasosPositivos.shape

(619278, 41)

# **Remove do Dataset os óbitos que ainda estão em investigação**

In [340]:
#remove os óbitos em investigação
dfCasosPositivosSemObitosEmInvestigacao = dfCasosPositivos.loc[~dfCasosPositivos.tipoObito.eq("Em investigação")]

In [341]:
#mostra o número de registros após a filtragem
dfCasosPositivosSemObitosEmInvestigacao.shape

(618995, 41)

# **Seleciona apenas os pacientes que possuem evolução do caso como "cura" ou "obito"** 

In [342]:
dfEvolucaoPositivos = dfCasosPositivosSemObitosEmInvestigacao.loc[(dfCasosPositivosSemObitosEmInvestigacao.evolucaoCaso.eq("Óbito.0")) | (dfCasosPositivosSemObitosEmInvestigacao.evolucaoCaso.eq("Cura.0"))]

In [343]:
dfEvolucaoPositivos.evolucaoCaso.value_counts(dropna=False)

Óbito.0    20104
Cura.0     13450
Name: evolucaoCaso, dtype: int64

# **Remove os registros que não possui a cor / raça do paciente** 

In [344]:
#remove os pacientes que não possuem a raça informada
dfEvoluidosComRaca= dfEvolucaoPositivos.dropna(subset=["racaCorPaciente"])

In [345]:
#mostra o número de registros após a filtragem
dfEvoluidosComRaca.shape

(22493, 41)

# **Remove os pacientes duplicados**

In [346]:
#Remove as linhas duplicadas
dfEvoluidosComRacaUniq = dfEvoluidosComRaca.drop_duplicates(subset=["codigoPaciente"], keep="last")

In [347]:
#mostra o número de registros após a filtragem
dfEvoluidosComRacaUniq.shape

(19928, 41)

# **Remove o único paciente sem idade**

In [348]:
#remove o único paciente que não possui idade
dfEvoluidosComRacaUniqIdade = dfEvoluidosComRacaUniq.dropna(subset=["idadePaciente"])

In [349]:
#mostra o número de registros após a filtragem
dfEvoluidosComRacaUniqIdade.shape

(19927, 41)

# **Altera os valores ignorados nas colunas comorbidades para Não**

In [350]:
#seleciona apenas as features que possuem comorbidades
comorbidade_cols = [col for col in dfEvoluidosComRacaUniqIdade.columns if 'comorbidade' in col]
#Substitui Ignorado e Não por 0 e Sim por 1
for c in comorbidade_cols:
  dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade[c].eq("Ignorado") | dfEvoluidosComRacaUniqIdade[c].eq("Não") | dfEvoluidosComRacaUniqIdade[c].isna() ,c] = 0
  dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade[c].eq("Sim"),c] = 1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


# **Cria uma coluna que agrupa a idade em categorias de 5 anos**

In [351]:
bins = [x for x in range(0,110, 10)]
labels = ["{} a {}".format(x, x+9) for x in range(0,100, 10)]
labels[-1] = 'mais de 90'
dfEvoluidosComRacaUniqIdade['idadeCategoria'] = pd.cut(dfEvoluidosComRacaUniqIdade['idadePaciente'], bins=bins, labels=labels)
#Prenche com a categoria mais de 90 anos quando as idades foram maiores que 100 (bins pega até de 90 a 100)
dfEvoluidosComRacaUniqIdade.idadeCategoria.fillna("mais de 90", inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


# **Cria a coluna tempo sobrevivência**

Essa coluna é criada com a espectativa de vida do paciente após os primeiros sintomas da doença.

In [352]:
#converte as datas do óbito para datetime
dfEvoluidosComRacaUniqIdade["dataObito"] = pd.to_datetime(dfEvoluidosComRacaUniqIdade['dataObito']).dt.date 
dfEvoluidosComRacaUniqIdade["dataInicioSintomas"] = pd.to_datetime(dfEvoluidosComRacaUniqIdade['dataInicioSintomas']).dt.date 
dfEvoluidosComRacaUniqIdade["dataNotificacao"] = pd.to_datetime(dfEvoluidosComRacaUniqIdade['dataNotificacao']).dt.date 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [353]:
import numpy as np
import datetime
#variável date possui a data de extração do dataset (última data de acompanhamento)
date = datetime.date(2021, 5, 25)
#cria uma nova coluna final_acompanhamento. Essa coluna vai possuir a data do óbito  copia o valor da real e preenche os dados faltantes com a data de aquisição do dataset
#essa data vai ser mascarada posteriormente pela regressão cox
dfEvoluidosComRacaUniqIdade["final_acompanhamento"] = dfEvoluidosComRacaUniqIdade["dataObito"].apply(lambda value: date if pd.isnull(value) else value )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [354]:
#cria uma nova coluna. Data de inicio do acompanhamento. 
#Utiliza como data de inicio do acomanhamento a data dos sintomas, ou em caso de nulidade a data de notificação
def fuc_inicio_acompanhamento (row):
  if pd.isnull(row['dataInicioSintomas']):     
    return row["dataNotificacao"]
  else:
    return row["dataInicioSintomas"]

dfEvoluidosComRacaUniqIdade["inicio_acompanhamento"] = dfEvoluidosComRacaUniqIdade.apply(lambda row: fuc_inicio_acompanhamento(row), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [355]:
#cria a coluna tempo de vida, que possui a quantidade de dias que o paciente está vivo após o inicio dos sintomas
dfEvoluidosComRacaUniqIdade["tempo_vida"] = dfEvoluidosComRacaUniqIdade["final_acompanhamento"] - dfEvoluidosComRacaUniqIdade["inicio_acompanhamento"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [356]:
#altera o tipo dias para int
dfEvoluidosComRacaUniqIdade["tempo_vida"] = dfEvoluidosComRacaUniqIdade["tempo_vida"].astype('timedelta64[D]')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [357]:
#remove os tempos de vida negativos
dfEvoluidosComRacaUniqIdade = dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.tempo_vida >= 0]

# **Converte o sexo do paciente para categórico númerico**


*   0 = Feminino
*   1 = Masculino



In [358]:
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.sexoPaciente.eq("FEMININO"),"sexo"] = 0
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.sexoPaciente.eq("MASCULINO"),"sexo"] = 1

In [359]:
#remove os sexos que não são masculinos ou femininos
dfEvoluidosComRacaUniqIdade.dropna(subset=["sexo"], inplace=True)

# **Converte a evolução do paciente para categórico númerico**


*  0 = Recuperado
*  1 = Óbito 

In [360]:
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.evolucaoCaso.eq("Óbito.0"),"target"] = 1
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.evolucaoCaso.eq("Cura.0"),"target"] = 0

# **Converte a raça do paciente para categórico númerico**


*   0 = Parda
*   1 = Branca
*   2 = Preta
*   3 = Amarela
*   4 = Indígina

In [361]:
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.racaCorPaciente.eq("Parda"),"racaCor"] = 0
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.racaCorPaciente.eq("Branca"),"racaCor"] = 1
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.racaCorPaciente.eq("Preta"),"racaCor"] = 2
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.racaCorPaciente.eq("Amarela"),"racaCor"] = 3
dfEvoluidosComRacaUniqIdade.loc[dfEvoluidosComRacaUniqIdade.racaCorPaciente.eq("Indígena"),"racaCor"] = 4

# **Renomeia as colunas comorbidades para melhorar a leitura**

In [362]:
#Melhora a visualização dos nomes das comorbidades
def ajustaNome(x):
  x = x.replace("comorbidade", "")
  return x.replace("Sivep", "")

cols_dic = {c:ajustaNome(c) for c in comorbidade_cols}
dfEvoluidosComRacaUniqIdade.rename(columns = cols_dic, inplace = True) 

# **Remove os pacientes que não são do Ceará - Que não foi informado o código do município**

In [363]:
dfEvoluidosComRacaUniqIdade.dropna(subset=["codigoMunicipioPaciente"], inplace=True)

In [364]:
#corrige o tipo do campo codigoMunicipio de float para int
dfEvoluidosComRacaUniqIdade["codigoMunicipioPaciente"] = dfEvoluidosComRacaUniqIdade["codigoMunicipioPaciente"].astype(int)


# **Abre dados do IBGE relativo aos município dos paciente**

In [365]:
#abre dataset IBGE
dfIbge = pd.read_csv("https://raw.githubusercontent.com/softip/projeto_IA368X/main/data/external/dados_ibge.csv")

In [366]:
#remove o último digito da coluna código do Ibge, para ficar igual ao dataset da covid
dfIbge['codigo_ibge_6d'] = dfIbge['codigo_ibge'].astype(str).str[:-1].astype(np.int64)

# **Junta os datasets com dados do IBGE e dados dos pacientes com COVID**

In [379]:
dfPacienteIBGE = pd.merge(dfEvoluidosComRacaUniqIdade, dfIbge, how="inner", left_on="codigoMunicipioPaciente", right_on="codigo_ibge_6d")

# **Seleciona as colunas a serem utilizadas no modelo**

In [369]:
features = ["sexo",
"idadePaciente",
"racaCor",
"Asma",
"Cardiovascular",
"Diabetes",
"Hematologia",
"Imunodeficiencia",
"Neurologia",
"Obesidade",
"Pneumopatia",
"Puerpera",
"Renal",
"SindromeDown",
"tempo_vida",
"area_territorial_km2",
"populacao_estimada_2020",
"densidade_demografica_hab_km2_2010",
"escolarizacao6a14 anos_2010",
"IDHM_2010",
"mortalidade_infantil_2019",
"PIB_per_capita_2018",
"target"]

In [380]:
dfFinal = dfPacienteIBGE.loc[:,features]

In [381]:
dfFinal.head()

,sexo,idadePaciente,racaCor,Asma,Cardiovascular,Diabetes,Hematologia,Imunodeficiencia,Neurologia,Obesidade,Pneumopatia,Puerpera,Renal,SindromeDown,tempo_vida,area_territorial_km2,populacao_estimada_2020,densidade_demografica_hab_km2_2010,escolarizacao6a14 anos_2010,IDHM_2010,mortalidade_infantil_2019,PIB_per_capita_2018,target
0,0.0,83.0,0.0,0,1,0,0,0,1,0,0,0,0,0,479.0,312.353,2686612,7786.44,96.1,0.754,11.67,25356.73,1.0
1,1.0,39.0,1.0,0,0,0,0,0,0,0,0,0,0,0,468.0,312.353,2686612,7786.44,96.1,0.754,11.67,25356.73,0.0
2,1.0,29.0,1.0,0,0,0,0,0,0,0,0,0,0,0,456.0,312.353,2686612,7786.44,96.1,0.754,11.67,25356.73,0.0
3,0.0,59.0,0.0,0,1,0,0,0,0,0,0,0,0,0,433.0,312.353,2686612,7786.44,96.1,0.754,11.67,25356.73,0.0
4,1.0,44.0,0.0,0,1,0,0,1,0,0,0,0,0,0,436.0,312.353,2686612,7786.44,96.1,0.754,11.67,25356.73,0.0


# **Verifica a quantidade de registros faltantes**

In [382]:
#mostra os campos e a quantidade de dados faltantes
dfFinal.isna().sum()

sexo                                  0
idadePaciente                         0
racaCor                               0
Asma                                  0
Cardiovascular                        0
Diabetes                              0
Hematologia                           0
Imunodeficiencia                      0
Neurologia                            0
Obesidade                             0
Pneumopatia                           0
Puerpera                              0
Renal                                 0
SindromeDown                          0
tempo_vida                            0
area_territorial_km2                  0
populacao_estimada_2020               0
densidade_demografica_hab_km2_2010    0
escolarizacao6a14 anos_2010           0
IDHM_2010                             0
mortalidade_infantil_2019             0
PIB_per_capita_2018                   0
target                                0
dtype: int64

# **Verifica os valores únicos para todas colunas**

In [383]:
for c in dfFinal.columns:
  print(c, ":", dfFinal[c].unique())

sexo : [0. 1.]
idadePaciente : [ 83.  39.  29.  59.  44.  45.  41.  70.  72.  48.  52.   3.  43.  67.
  64.  86.  66.  77.  60.  98.  36.  76.  61.  69.  42.  53.  55.  65.
  78.  79.  49.  58.  63.  38.  75.  74.  57.  15.  50.  47.  81.  90.
  68.  82.  46.  71.  87.  92.  73.  56.  62.  89.  40.  88.  23.  35.
  93.  95.  51.  84.  85.  54.  94.  34. 100.  80.  97.   4.  91.  24.
   0.  33.  37.  20.  96.  19. 109. 104.  18.  28.  27.  31.  17.  21.
  14.  32.  13.  26.  16.  30.  25.  99. 110.  12.   1.   6.   2.   5.
   8.  10. 101.  22.   9. 102. 105. 114. 103. 106.   7.  11. 107. 108.]
racaCor : [0. 1. 2. 3. 4.]
Asma : [0 1]
Cardiovascular : [1 0]
Diabetes : [0 1]
Hematologia : [0 1]
Imunodeficiencia : [0 1]
Neurologia : [1 0]
Obesidade : [0 1]
Pneumopatia : [0 1]
Puerpera : [0 1]
Renal : [0 1]
SindromeDown : [0 1]
tempo_vida : [479. 468. 456. 433. 436. 438. 443.  11. 431. 439. 429. 430. 437.   8.
  22. 432.  13. 434.   1.  35. 428.  15. 427.  30.  41.  19.  27.  12.
 426.  20. 

# **Corrige valor da coluna mortalidade_infantil_2019**

A coluna é númerica, mas existe municípios que não possuem valor de mortalidade. Vamos substituir - por 0 e alterar o tipo para float

In [389]:
dfFinal.loc[dfFinal["mortalidade_infantil_2019"].eq("-"), "mortalidade_infantil_2019"] = 0

# **Mostra a distribuição dos dados**

In [391]:
dfFinal.target.value_counts()

1.0    11763
0.0     8071
Name: target, dtype: int64

# **Salva o dataset**

In [392]:
dfFinal.to_csv("DatasetCovidTratadoCox.csv", index=False)